# CNOT gate
The Controlled NOT gate operates on two qubits. The second qubit (**target**) is flipped if and only if the first qubit (**control**) is |1⟩.

From a classical computing viewpoint the output of a CNOT gate can be (improperly) described as:
CNOT|a, b⟩ = |a, a ⊕ b⟩
where ⊕ represents the XOR boolean operation.


| Initial State | Final State |
| :---:  | :---: |
| &#124;00⟩ | &#124;00⟩ |
| &#124;01⟩ | &#124;01⟩ | 
| &#124;10⟩ | &#124;11⟩ |
| &#124;11⟩ | &#124;10⟩ |
    

In [1]:
from functools import reduce
from itertools import product
from math import log

import numpy as np
np.set_printoptions(precision=3, suppress=True, floatmode='fixed', sign='+')

In [2]:
# CNOT is a 4x4 matrix
Cx = np.array([[1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, 0, 1],
               [0, 0, 1, 0]], dtype=np.complex64)

zero_qubit = np.array([1, 0]).T
one_qubit = np.array([0, 1]).T

plus_qubit = 1 / np.sqrt(2) * np.array([1, 1]).T
minus_qubit = 1 / np.sqrt(2) * np.array([1, -1]).T

cw_qubit = 1 / np.sqrt(2) * np.array([1, 1.0j]).T
ccw_qubit = 1 / np.sqrt(2) * np.array([1, -1.0j]).T

# from quantum_registers notebook
def create_quantum_state(qubits):
    return reduce(lambda x,y: np.kron(x, y), qubits)

In [3]:
a = create_quantum_state([zero_qubit, zero_qubit])
print("CNOT|00⟩ =", Cx @ a)

b = create_quantum_state([zero_qubit, one_qubit])
print("CNOT|01⟩ =", Cx @ b)

c = create_quantum_state([one_qubit, zero_qubit])
print("CNOT|10⟩ =", Cx @ c)

d = create_quantum_state([one_qubit, one_qubit])
print("CNOT|11⟩ =", Cx @ d)

CNOT|00⟩ = [+1.000+0.000j +0.000+0.000j +0.000+0.000j +0.000+0.000j]
CNOT|01⟩ = [+0.000+0.000j +1.000+0.000j +0.000+0.000j +0.000+0.000j]
CNOT|10⟩ = [+0.000+0.000j +0.000+0.000j +0.000+0.000j +1.000+0.000j]
CNOT|11⟩ = [+0.000+0.000j +0.000+0.000j +1.000+0.000j +0.000+0.000j]


## Entanglement

What happens when the CNOT is applied to a quantum state created from the the qubits |+⟩ and |0⟩?

CNOT|Ψ⟩ = $\frac{1}{\sqrt{2}}$ (CNOT|00⟩ + CNOT|10⟩) = $\frac{1}{\sqrt{2}}$ (|00⟩ + |11⟩)

As it was discussed in the quantum_registers notebook this state is non separable. In other words, it is an **entangled state**.

In [4]:
psi = create_quantum_state([plus_qubit, zero_qubit])
out = Cx @ psi

print("CNOT|Ψ⟩ =", out)

CNOT|Ψ⟩ = [+0.707+0.000j +0.000+0.000j +0.000+0.000j +0.707+0.000j]


In [5]:
def guess_qubits(quantum_state):
    v = [zero_qubit, one_qubit, plus_qubit, minus_qubit, cw_qubit, ccw_qubit]
    n_qubits = int(log(quantum_state.size, 2))
    
    # product computes the cartesian product of the input iterables
    # product(v, repeat=3) is the same as product(v, v, v)
    for qubits in product(v, repeat=n_qubits):
        guess = create_quantum_state(qubits)
        
        # check if the guessed state and the input are element-wise equal
        if np.allclose(guess, quantum_state):
            return qubits


print(guess_qubits(out))

None


In [6]:
from random import random

def measure(state):
    n = int(log(state.size, 2))
    
    # element-wise product
    probabilities = state.conj() * state
    
    rand = random()
    for idx, realization in enumerate(product([0, 1], repeat=n)):
        if rand < sum(probabilities[0:(idx+1)]):
            return "|" + "".join(map(str, realization)) + "⟩"

N = 1000
occurrences = dict()
for _ in range(N):
    m = measure(out)
    if m not in occurrences:
        occurrences[m] = 0
    occurrences[m] += 1

for k,v in occurrences.items():
    print(k, " with probability ", v * 100.0 / N)

|11⟩  with probability  49.1
|00⟩  with probability  50.9
